In [8]:
import gym
import math
import numpy as np
import torch
from torch import nn
from random import sample
 

In [3]:
class Agent:

    def __init__(self, n_states, n_actions, epsilon, learning_rate, discount_value):
        self.learning_rate = learning_rate
        self.discount_value = discount_value
        self.n_states = n_states
        self.n_actions = n_actions
        self.epsilon = epsilon
        self.initialize_q_table(n_states, n_actions)
    
    
    # This is the policy
    def choose_best_action(self, state):
        '''parameters: state 
        returns: action'''
        # Always need choose_action
        return np.argmax(self.q_table[state]) # <-- returns the index with the highest action value for that state
    
    def choose_action(self, state): # Policy
        if np.random.rand(1) < self.epsilon:
            action = np.random.randint(0, self.n_actions)
        else:
            action = self.choose_best_action(state)
            
        return action
    
        
    def initialize_q_table(self, n_states, n_actions):
        self.q_table = np.zeros((n_states, n_actions))
        
       
    
    # HOMEWORK
    def bellman_equation(self, state, action, next_state, reward):
        current_q = self.q_table[state][action]
        max_q_next_state = np.max(self.q_table[next_state])
        new_q =  current_q + self.learning_rate * (reward + (self.discount_value * max_q_next_state) - current_q)
        # single step forecasting?
        # value of state = current value + immediate reward + negative or positive expected reward for that state
        # reward - immediate reward for moving to next_state? 0 for everything except the goal node (1)
        # max_q_next_state - max value of state for all of its actions?
        return new_q
        
    
    def update_q_table(self, state, action, next_state, reward):
        new_q = self.bellman_equation(state, action, next_state, reward)
        self.q_table[state][action] = new_q
        
    
        


In [15]:
class FCNeuralNetwork(nn.Module):
    
    def __init__(self, x_size, y_size, hidden_size):
        super.__init__()
        
        self.linear_relu_nn = nn.Sequential(
            nn.Linear(x_size, hidden_size),
            nn.ReLU()
            nn.Linear(hidden_size, hidden_size)
            nn.ReLU()
            nn.Linear(hidden_size, y_size)
        )
        
        
    def forward(self, x):
        logits = self.linear_relu_nn(x)
        return out
        
    
    
class ReplayBuffer:
    
    def __init__(self, limit):
        self.buffer = deque([], maxlen = limit)
        
    def append(self, transition):
        self.buffer.append(transition)
        
    def sample(self, sample_size):
        return sample(self.buffer, sample_size)
    
    def __len__(self):
        return len(self.buffer)
    
    
 
    

class DQNAgent(Agent): 
    
    # Hyperparameters
    self.discount_factor = 0.95
    
    def __init__(self,  num_actions, num_states, model, replay_buffer)
    
    self.num_states = num_states,
    self.num_actions = num_actions
    self.model = model
    self.replay_buffer = replay_buffer
    
    
    # Target model initialisation
    self.target_model = model
    self.target_model.set_weights = self.model.get_weights()
    
    def choose_best_action(self, state):
        # pass through network
        action = max(self.model(state))
        return action
    
    def choose_action(self):
        if np.random.rand(1) < self.epsilon:
            action = np.random.randint(0, self.n_actions)
        else:
            action = max(self.model(state))
        return action
    
    def learn(self, state, action, next_state, reward):
        # Q value predicted for that state, action pair
        predicted_value = self.model(state)[action] # predicted value = Q(s, a)
        
        target_value = reward + self.discount_factor * max(self.model(next_state))  # target value = r + γQ(s', a')
        
        
    
    
## TO DO
# FIGURE OUT LOSS FUNCTION AND BACKPROPOGATION
# IMPLEMENT OPTIONAL BUFFER SAMPLING
# IMPLEMENT OPTIONAL TARGET NETWORK
# USE MATPLOTLIB OR SOMETHING TO VISUALIZE THE PERFORMANCE OF THE THREE AGENT TYPES

    
    
    

SyntaxError: invalid syntax. Perhaps you forgot a comma? (2529935670.py, line 8)

In [ ]:
loss_function = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)



In [11]:
episodes = 3000

# Epsilon stuff
start_epsilon = 0.3
min_epsilon = 0.1
epsilon_reduction = 0.05
epsilon_decay = 100

# Visualization
render_decay = 1000

# Environment setup
env_name = "FrozenLake-v1"
env = gym.make(env_name, render_mode="human")
num_actions = env.action_space.N
num_states = env.observation_space.N

seed = 42
np.random.seed(seed)


# Q-Table hyperparameters
learning_rate = 0.1
discount_rate = 0.95


# Neural Network hyperparameters
hidden_size = 128

# Neural network setup
model = FCNeuralNetwork(num_actions, num_states, hidden_size)

# Replay buffer setup
replay_buffer = ReplayBuffer()

# Agent setup
agent = DQN_Agent(num_actions, num_states, model)

for episode in range(episodes):
    
    
    

SyntaxError: incomplete input (712806370.py, line 37)